In [ ]:
import glob
import json
import os

import numpy as np
import cv2
import torch

model = torch.hub.load('pytorch/vision:v0.5.0', 'resnet18', pretrained=True)
from torch import nn
import torchvision
from torch.utils.data import Dataset, DataLoader

In [ ]:
model.fc = nn.Linear(in_features=512, out_features=2)

In [ ]:
model

In [ ]:
data_path = '/root/data/alok/biomass_estimation/playground/akpr'

In [ ]:
class CropDataset(Dataset):
    
    def __init__(self, data_path, transform=None):
        self.images = sorted(glob.glob(os.path.join(data_path, 'images/*.jpg')))
        self.labels = sorted(glob.glob(os.path.join(data_path, 'labels/*.json')))
        self.transform = transform
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        im = cv2.imread(self.images[idx]) / 255.0
        label = np.array(json.load(open(self.labels[idx])))
        sample = {
            'image': im,
            'label': label
        }
        
        if self.transform:
            sample = self.transform(sample)
        
        return sample
    
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, labels = sample['image'], sample['labels']
        return {'image': image_tensor,
                'labels': torch.from_numpy(np.array(labels))}
        

In [ ]:
dataset = CropDataset(data_path)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True, num_workers=1)
epochs = 1000

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

for e in range(epochs):
    total_loss = 0.0
    for batch in dataloader:
        X, y = batch['image'], batch['label']
        X = X.view(1, 3, 224, 224)
        y_pred = model(X.float())
        
        loss = criterion(y, y_pred)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
    print(total_loss / len(dataloader))
        
    
    